In [126]:
#creation database

import sqlite3

# connexion (+ creation de la base si elle n'existe pas)
connexion = sqlite3.connect('Database.db')
cursor = connexion.cursor()
# Creation des tables dans la base de donnée
cursor.executescript('''
    CREATE TABLE IF NOT EXISTS d_entreprise (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        entreprise_nom TEXT,
        entreprise_description TEXT
    );

    CREATE TABLE IF NOT EXISTS h_departement (
        departement TEXT PRIMARY KEY
    );
    
    CREATE TABLE IF NOT EXISTS d_ville (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        ville TEXT,
        latitude REAL,
        longitude REAL,
        code_postal TEXT,
        departement TEXT,
        FOREIGN KEY (departement) REFERENCES h_departement (departement)
    );

    CREATE TABLE IF NOT EXISTS offres (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        intitule TEXT,
        description_offre TEXT,
        date_creation TEXT,
        salaire TEXT,
        qualification_libelle TEXT,
        experience TEXT,
        type_contrat TEXT,
        secteur_activite TEXT,
        id_ville INTEGER,
        id_entreprise INTEGER,
        FOREIGN KEY (id_ville) REFERENCES d_ville (id),
        FOREIGN KEY (id_entreprise) REFERENCES d_entreprise (id)
    );
''')
# Commit changes and close the connection
connexion.commit()
connexion.close()

In [127]:
#connexion.close()

In [128]:
import pandas as pd

In [129]:
df = pd.read_csv('Jobs.csv')

In [130]:
df.head(3)

,intitule,description_offre,dateCreation,typeContrat,experienceExige,secteurActiviteLibelle,qualificationLibelle,latitude,longitude,codePostal,commune,departement,ville,experienceOuiNon,entreprise_nom,entreprise_description,salaire,profils
0,Consultant(e) Senior Data Engineer F/H - ADN E...,Poste et missions\n \n \n \n- \nParticiper à...,30/12/2023,CDI,E,Activités des agences de travail temporaire,NaN,NaN,NaN,NaN,NaN,Ile-de-France,NaN,oui,ADN Emploi,ADN Emploi Lille est une agence de recrutement...,NaN,NaN
1,Senior Data Engineer F/H (H/F),"Le saviez-vous ?\n\nNous rejoindre, c'est rejo...",29/12/2023,CDI,E,Autres activités informatiques,NaN,48.727791,2.275131,91300.0,91377.0,91,MASSY,oui,NaN,"Nous rejoindre, c'est rejoindre l'un des leade...",NaN,NaN
2,Data Engineer H/F,"Rejoindre le Groupe IRCEM, c'est participer à ...",29/12/2023,CDI,E,NaN,NaN,50.688454,3.183662,59100.0,59512.0,59,ROUBAIX,oui,IRCEM,NaN,NaN,NaN


In [131]:
df = df.rename(columns={'dateCreation':'date_creation', 'qualificationLibelle':'qualification_libelle', 'experienceOuiNon':"experience", 'typeContrat':'type_contrat', 'secteurActiviteLibelle':'secteur_activite', 'codePostal':'code_postal' })

In [132]:
df.columns

Index(['intitule', 'description_offre', 'date_creation', 'type_contrat',
       'experienceExige', 'secteur_activite', 'qualification_libelle',
       'latitude', 'longitude', 'code_postal', 'commune', 'departement',
       'ville', 'experience', 'entreprise_nom', 'entreprise_description',
       'salaire', 'profils'],
      dtype='object')

In [133]:
df['entreprise_nom'].unique().shape

(172,)

In [134]:
df['entreprise_description'].unique().shape

(200,)

In [135]:
df[['entreprise_nom', 'entreprise_description']].drop_duplicates()

,entreprise_nom,entreprise_description
0,ADN Emploi,ADN Emploi Lille est une agence de recrutement...
1,NaN,"Nous rejoindre, c'est rejoindre l'un des leade..."
2,IRCEM,NaN
4,CPF,"Depuis 2002, Amorino émerveille les gourmands ..."
5,NaN,Ministère de l'Intérieur
...,...,...
423,EURO-INFORMATION DEVELOPPEMENTS,"Euro-Information, filiale technologique de Cré..."
424,CNAF,Vous souhaitez rejoindre une équipe expériment...
425,DIRECTION DU NUMÉRIQUE DES MINISTÈRES SOCIAUX,Vous souhaitez intégrer une Direction numériqu...
426,CNAF,Vous souhaitez rejoindre une équipe expériment...


In [136]:
connexion = sqlite3.connect('Database.db')

# insertion des données dans les tables d_entreprise, h_departement, d_ville
# on insert pas les id pour le moment
# d_entreprise
print(df[['entreprise_nom', 'entreprise_description']].drop_duplicates().head(3))
df[['entreprise_nom', 'entreprise_description']].drop_duplicates().to_sql('d_entreprise', connexion, if_exists='append', index=False)

# h_departement
print(df[['departement']].drop_duplicates()) #on pourra ajouter la région dans cette table plus tard
df[['departement']].drop_duplicates().to_sql('h_departement', connexion, if_exists='append', index=False)

# d_ville
print(df[['ville', 'latitude', 'longitude', 'code_postal', 'departement']].drop_duplicates().head(3))
df[['ville', 'latitude', 'longitude', 'code_postal', 'departement']].drop_duplicates().to_sql('d_ville', connexion, if_exists='append', index=False)


  entreprise_nom                             entreprise_description
0     ADN Emploi  ADN Emploi Lille est une agence de recrutement...
1            NaN  Nous rejoindre, c'est rejoindre l'un des leade...
2          IRCEM                                                NaN
          departement
0       Ile-de-France
1                  91
2                  59
4                  94
5                  92
7                  45
8                  13
9                  69
11                 44
12                 33
14                 67
15                 93
16                 14
17                 34
18                 75
23                 31
31                 73
56                 26
64                 74
67   Pays de la Loire
71                 06
76                 78
80                 84
86                 35
95                 76
139                79
140            France
153                38
156                80
160                83
161                40
169                64
18

190

In [137]:
# d_ville
print(df[['ville', 'latitude', 'longitude', 'code_postal', 'departement']].drop_duplicates().head(3))
df[['ville', 'latitude', 'longitude', 'code_postal', 'departement']].drop_duplicates().to_sql('d_ville', connexion, if_exists='append', index=False)

     ville   latitude  longitude  code_postal    departement
0      NaN        NaN        NaN          NaN  Ile-de-France
1    MASSY  48.727791   2.275131      91300.0             91
2  ROUBAIX  50.688454   3.183662      59100.0             59


190

In [139]:
''' la fonction map est comme la fonction apply de R, 
il applique une fonction sur chaque ligne de la dataframe.
Nous nous en servons pour réccupérer les id de chaque entreprise dans d_entreprise,
ces id seront placés dans la colonne id_entreprise de la table offres
'''
# creation de id_entreprise
company_mapping = pd.read_sql('SELECT id,entreprise_nom,entreprise_description FROM d_entreprise', connexion)
df['id_entreprise'] = df['entreprise_nom'].map(company_mapping.set_index(['entreprise_nom','entreprise_description'])['id'])


# Map company names to company IDs (drop duplicates first)
#df['id_entreprise'] = df['entreprise_nom'].map(
#    df[['entreprise_nom', 'id_entreprise']].drop_duplicates().set_index('entreprise_nom')['id_entreprise']
#)

In [140]:
df['id_entreprise'].unique()

array([nan])

In [ ]:
# fin temporaire de l'intégration des données
connexion.commit()
connexion.close()


In [144]:
connexion = sqlite3.connect('Database.db')
connexion.close()

In [145]:
# accès à la table departement pour afficher son contenu
connexion = sqlite3.connect('Database.db')
cursor = connexion.cursor()

# selectionner toutes les colonnes de la table d_ville
query = 'SELECT * FROM d_ville'
cursor.execute(query)

# placer la selection dans une variable
rows = cursor.fetchall()

# parcourir la variable pour afficher les données
for row in rows:
    print(row)

connexion.close()

(1, None, None, None, None, 'Ile-de-France')
(2, 'MASSY', 48.727791, 2.275131, '91300.0', '91')
(3, 'ROUBAIX', 50.688454, 3.183662, '59100.0', '59')
(4, 'ORLY', 48.742287, 2.395245, '94310.0', '94')
(5, 'Hauts de Seine', None, None, None, '92')
(6, 'ORLEANS', 1.908606, 47.902733, '45000.0', '45')
(7, 'AIX EN PROVENCE', 5.447473, 43.529842, '13080.0', '13')
(8, 'LYON 09', 4.806248, 45.774, '69009.0', '69')
(9, 'LESQUIN', 3.10997, 50.5888, '59810.0', '59')
(10, 'NANTES', -1.554136, 47.218637, '44000.0', '44')
(11, 'PESSAC', -0.630839, 44.805615, '33600.0', '33')
(12, 'BEAURAIN', 3.554142, 50.18045, '59730.0', '59')
(13, 'SCHILTIGHEIM', 7.748448, 48.604714, '67300.0', '67')
(14, 'MONTREUIL', 2.441218, 48.862335, '93100.0', '93')
(15, 'CAEN', -0.363561, 49.18134, '14000.0', '14')
(16, 'MONTPELLIER', 3.876733, 43.611242, '34000.0', '34')
(17, 'PARIS 01', 48.859, 2.347, '75001.0', '75')
(18, 'NANTES', 47.239367, -1.555335, '44000.0', '44')
(19, 'Paris (Dept.)', None, None, None, '75')
(20, '

In [141]:
# Insertion des données dans la table offre
#df[['intitule', 'description_offre', 'date_creation', 'salaire', 'qualification_libelle',
#    'experience', 'type_contrat', 'secteur_activite', 'id_ville', 'id_entreprise']].to_sql('offres', connexion, if_exists='append', index=False)

# Commit et fermeture de la connexion
#connexion.commit()
#connexion.close()


KeyError: "['id_ville'] not in index"

In [ ]:
# probleme avec id entreprise, complexe à réccupérer